In [1]:
!python --version

Python 3.9.12


In [2]:
import os
os.getcwd()

'C:\\Users\\Notebiz003'

In [3]:
!pip install pandas
!pip install konlpy
!pip install umap
!pip install -U scikit-learn
!pip install --upgrade pip
!pip install matplotlib

In [6]:
import pandas as pd
path = "C:/Users/Notebiz003/BerTopic/data/"
file_list = os.listdir(path)
dfs = []

In [7]:
for i in file_list:
    df = pd.read_excel(path+i)
    dfs.append(df)
    print(i)

C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20180101-20181213.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20190101-20191231.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20200101-20201231.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20210101-20211231.xlsx


C:\Users\Notebiz003\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20220101-20220812.xlsx


In [8]:
from datetime import datetime

import logging
import re

import numpy as np
import pandas as pd

In [9]:
## 데이터 전처리
data = pd.concat(dfs)
data = data[['뉴스 식별자','일자','제목','키워드','특성추출(가중치순 상위 50개)']]
cluster_mode = '특성추출(가중치순 상위 50개)'
corpus = data[cluster_mode].values.tolist()

for i, cp in enumerate(corpus):
    corpus[i] = cp.split(',')
df = data[['일자']]
df['키워드'] = corpus
df['일자'] = [str(x)[0:4] for x in df['일자']] 

# 키워드 사전 
keyword_vocab = []
for i in df['키워드']:
    keyword_vocab.extend(i)
keyword_vocab = list(set(keyword_vocab))

# 연도별 카운팅 후 병합
from collections import Counter
years = ['2018','2019','2020','2021','2022']

tmp_df_list = []
for year in years:
    print(year)
    tmp = []
    for i in df[df['일자']== year]['키워드']:
        tmp.extend(i)
    tmp_df = pd.DataFrame(pd.Series(Counter(tmp)))
    tmp_df = tmp_df.reset_index()
    tmp_df = tmp_df.rename(columns = {0:'cnt','index':'keyword'})
    tmp_df_list.append(tmp_df)
    
df = tmp_df_list[0].merge(tmp_df_list[1], on = 'keyword', how = 'outer')
for i in range(len(tmp_df_list)):
    if i > 1:
        df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
col = ['keyword'] + years
df.columns = col

# 2022년 수치 날짜 보정
df['2022'] = round(df['2022']*(365/224)) 
df

# 합산 값 만들기
df.index = df['keyword']
df = df.drop('keyword', axis = 1)
df['tot_cnt'] = df.sum(axis = 1)
df = df.T
df['year_cnt'] = df.sum(axis = 1)
df = df.T
df

# 결측치 처리
df =df.fillna(0)
df

<ipython-input-9-49009ab8ddea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['키워드'] = corpus
<ipython-input-9-49009ab8ddea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['일자'] = [str(x)[0:4] for x in df['일자']]


2018
2019
2020
2021
2022


<ipython-input-9-49009ab8ddea>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')


,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
신한,473.0,453.0,513.0,623.0,629.0,2691.0
신한은행,633.0,605.0,621.0,979.0,904.0,3742.0
소비자,572.0,953.0,1244.0,1715.0,1361.0,5845.0
금융업,175.0,247.0,367.0,383.0,424.0,1596.0
재정의,15.0,1.0,2.0,5.0,2.0,25.0
...,...,...,...,...,...,...
호랑이띠,0.0,0.0,0.0,0.0,2.0,2.0
닛케이225지수,0.0,0.0,0.0,0.0,2.0,2.0
항공우주국,0.0,0.0,0.0,0.0,2.0,2.0


In [10]:
# 데이터 확인
key = 'nft'
df[df.index == key]

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
nft,0.0,0.0,7.0,356.0,753.0,1116.0


In [11]:
# 데이터 비교1 . 회귀선의 기울기
# 연도별 건수 보정(normaliae)

df2 = df.copy()
for year in years:
    df2[year] = df2[year][:-1]*10000 / df2[year][-1]
from scipy import stats

x = [1,2,3,4,5]

slopes = []
for i in list(df2.values):
    y = i[:-1]
    slope, intercept, r, p, std_err = stats.linregress(x, y)
    slopes.append(slope)
df2['slope'] = slopes

In [12]:
df2.sort_values(by = 'slope',ascending = False)[0:10]

,2018,2019,2020,2021,2022,tot_cnt,slope
keyword,,,,,,,
코로나19,0.000000,0.000000,130.589979,79.573891,49.051067,8598.0,17.767602
가상자산,0.058143,0.993242,6.469357,18.817016,32.512234,2048.0,8.273196
메타버스,0.000000,0.039730,0.000000,21.109729,30.646315,1840.0,8.236263
마이데이터,1.860573,4.131887,18.107697,27.009269,27.423363,2629.0,7.400296
빅테크,0.116286,1.430268,18.952939,26.617830,24.059054,2424.0,7.307310
esg,0.058143,1.311079,5.038946,24.688597,24.567941,1941.0,7.239711
코로나,0.000000,0.000000,54.810731,29.693420,21.231903,3499.0,7.215723
온라인,31.397174,28.605369,62.255367,58.631922,42.887878,6789.0,5.300796
nft,0.000000,0.000000,0.227565,9.953726,21.288446,1116.0,5.253062


In [13]:
df2.sort_values(by = 'slope',ascending = True)[0:50]

,2018,2019,2020,2021,2022,tot_cnt,slope
keyword,,,,,,,
블록체인,62.096634,39.531031,20.870990,21.361368,22.108319,4251.0,-9.814629
블록체,54.189197,43.623188,22.041326,25.695153,27.027562,4583.0,-7.225130
빅데이터,50.002907,42.590216,45.578081,31.287134,20.949188,5194.0,-6.941052
핀테크,49.072621,71.831260,47.853734,43.533573,34.971856,6918.0,-5.649922
서울,71.050642,70.122884,55.525936,47.503879,57.673877,8434.0,-4.937254
본부장,41.630327,37.782925,30.201168,31.454894,24.935470,4603.0,-3.971775
국민은행,25.989883,27.691586,16.417212,15.853266,12.298106,2651.0,-3.922187
kb,30.641316,32.101581,23.926867,20.690330,19.083268,3486.0,-3.452735
중국,34.595035,36.988331,32.411802,30.168738,21.458075,4361.0,-3.309351


In [14]:
# 시각화 비교 남는 시간 방법
x = []
y = []
keys = []
min_cnt = 300
for i in keyword_vocab:
    word = i
    if df['2021'][word] > min_cnt and df['2022'][word] > min_cnt:
        conf_2021 = df['2021'][word]/df['2021'][-1]
        conf_2022 = df['2022'][word]/df['2022'][-1]
        ratio_2022 = conf_2022 / conf_2021
        ratio_2021 = conf_2021 / conf_2022
        key = word
        x.append(ratio_2021)
        y.append(ratio_2022)
        keys.append(key)

In [15]:
df3 = pd.DataFrame({'key2021':x,'key2022':y}, index = keys)

In [16]:
df3 = df3.dropna()

In [18]:
import numpy as np
df3 = df3.dropna()
df3 = df3[df3.sum(axis = 1) != np.inf]

In [19]:
df2.sum(axis = 1)[0]

2786.3766134287994

In [20]:
df3.sort_values(by = 'key2021',ascending = False).head(10)

,key2021,key2022
비트코인,2.289220,0.436830
중앙은행,1.837033,0.544356
코로나19,1.622266,0.616422
농협은행,1.552466,0.644137
빅데이터,1.493477,0.669578
금융서비스,1.478539,0.676343
비대면,1.455396,0.687098
은행장,1.416376,0.706027
카카오페이,1.411917,0.708257
중국,1.405939,0.711269


In [21]:
df3.sort_values(by = 'key2022',ascending = False).head(10)

,key2021,key2022
nft,0.467565,2.138741
가상자산,0.578767,1.727810
간담회,0.616238,1.622749
부산,0.667042,1.499156
메타버스,0.688818,1.451763
메타,0.734272,1.361894
위원장,0.742653,1.346524
부동산,0.755977,1.322791
고객들,0.774894,1.290500
자회사,0.786145,1.272030


In [22]:
# 데이터 확인
key = '농협은행'
df[df.index == key]

,2018,2019,2020,2021,2022,tot_cnt
keyword,,,,,,
농협은행,290.0,427.0,531.0,540.0,344.0,2132.0


In [206]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer


from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

RepositoryNotFoundError: 401 Client Error: Repository Not Found for url: https://huggingface.co/api/models/kpfSBERT. If the repo is private, make sure you are authenticated. (Request ID: 1a1KF1SNh-lUdjb_2pT-o)

In [209]:

model = SentenceTransformer(model_path) 

In [218]:
for idx, i in enumerate(corpus):
    corpus[idx] = ','.join(i)

In [223]:
len(corpus)

68315

In [224]:
# BERT 임베딩
corpus_embeddings = model.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/2135 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [171]:
len(corpus_embeddings)

68885

In [172]:
clustering = DBSCAN(eps=0.4, min_samples=10,metric = 'cosine').fit(corpus_embeddings)

In [225]:
set(clustering.labels_)

NameError: name 'clustering' is not defined

In [228]:
clustering_df = pd.DataFrame({'corpus':corpus,'corpus_e':corpus_embeddings,'Topic':clustering.labels_})

NameError: name 'clustering' is not defined

In [ ]:
# 클러스터별로 Topic 모델링 일반적인 시각화

In [ ]:
# T-sne 시각화

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [62]:
tf_idf, count = c_tf_idf(corpus, m=len(corpus))

<ipython-input-61-0f534e09bd9e>:7: RuntimeWarning: invalid value encountered in true_divide
  tf = np.divide(t.T, w)


In [66]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, corpus, n=20)

AttributeError: 'list' object has no attribute 'Topic'

In [ ]:
topic_sizes = extract_topic_sizes(rslt[category_id]); topic_sizes.head(10)

In [60]:
''' JAVA HOME 문제 해결 필요
from konlpy.tag import Kkma
 
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result
    
custom_tokenizer = CustomTokenizer(Kkma())
'''

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
 model.get_topic_freq().head()

In [ ]:
 model.get_topic(49)

In [ ]:
model.visualize_topics(

In [ ]:
model.visualize_distribution(probabilities[0])